In [1]:
import os, csv
import cv2
import glob
import numpy as np
import numpy.random as nrand
import random
import math

In [2]:
autti_images_data = {}
cars_count = 0

with open('datasets/object-dataset/labels.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in reader:
        fname = row [0]
        if (row[6] == '"car"'):
            x1 = int(row [1])
            y1 = int(row [2])
            x2 = int(row [3])
            y2 = int(row [4])
            fname = os.path.join('datasets/object-dataset', fname)
#             img = cv2.imread(fname)
#             img_crop = img[y1:y2, x1:x2]
#             img_crop = cv2.resize(img_crop, (64,64))
#             cv2.imwrite(os.path.join('datasets/autti-vehicles', str(cars_count) + '.png'), img_crop)
            cars_count += 1
        if cars_count>10000:
            break
print 'cars_count = ',cars_count

cars_count =  25001


In [3]:
# imrotate_without_crop
def imrotate(img, angleInDegree):
    height = img.shape[0]
    width = img.shape[1]
    center = (int(img.shape[1]*0.5),int(img.shape[0]*0.5))
    # compute a rotation matrix with respect to the center of the image
    rot = cv2.getRotationMatrix2D(center, angleInDegree, 1.0)

    radians = math.radians(angleInDegree)
    sin = math.sin(radians)
    cos = math.cos(radians)
    bound_w = int((height * abs(sin)) + (width * abs(cos)))
    bound_h = int((height * abs(cos)) + (width * abs(sin)))

    rot[0, 2] += ((bound_w / 2) - center[0])
    rot[1, 2] += ((bound_h / 2) - center[1])

    img_rot = cv2.warpAffine(img, rot, (bound_w, bound_h), borderMode=cv2.BORDER_REFLECT_101, borderValue=(255,255,255))
    return img_rot

In [4]:
num_non_cars = 11000
image_names = glob.glob('datasets/manual-collection/*.png')
num_images = len(image_names)
num_crops_per_image = num_non_cars/num_images
maxRotAngleInDegrees = 10

count = 0
for fname in image_names:
    img = cv2.imread(fname)
    H,W = img.shape[:2]
    for x in range(num_crops_per_image):
        x1 = random.randint(0, W/2)
        x2 = random.randint(W/2, W-1)
        y1 = random.randint(0, H/2)
        y2 = random.randint(H/2, H-1)
        img_crop = img[y1:y2, x1:x2]
        if img_crop.shape[0]<50 or img_crop.shape[1]<50:
            continue
        img_crop = imrotate(img_crop, random.randint(-maxRotAngleInDegrees,maxRotAngleInDegrees))
        img_crop = cv2.resize(img_crop, (64,64))
        cv2.imwrite(os.path.join('datasets/manual-non-vehicles', str(count) + '.png'), img_crop)
        count += 1
    
#         if count>100:
#             break
        
    